<a href="https://colab.research.google.com/github/SCCSMARTCODE/Deep-Learning-01/blob/main/Car_and_Truck_Object_Tracking/TrackingVehicle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 🔧 **Project Setup and Environment**
- [x] Install necessary libraries and dependencies:
  - [x] Install **Ultralytics YOLOv8** with `pip install ultralytics`
  - [x] Install **OpenCV** with `pip install opencv-python`
  - [x] Install **Deep SORT** tracker and dependencies

### 📂 **Data Preparation**
- [x] Download or create a simple vehicle dataset (if fine-tuning YOLOv8 is needed):
  - [x] Label the dataset in **YOLO format** (class, x_center, y_center, width, height).
  - [x] Use tools like **LabelImg** or **Roboflow** for labeling if needed.

### 🏗 **Model Fine-Tuning with YOLOv8**
- [x] Load the **YOLOv8 pre-trained model** (e.g., `yolov8n.pt` or `yolov8s.pt`).
- [x] Fine-tune the model on your vehicle dataset:
  - [x] Prepare the dataset in the YOLOv8 compatible format.
  - [x] Use the Ultralytics API or command-line tool to fine-tune:
    - Command example: `yolo task=detect mode=train model=yolov8n.pt data=your_dataset.yaml epochs=50 imgsz=640`
  - [x] Check **training metrics** (loss, mAP) to validate improvements.
- [x] Save the fine-tuned model for inference.

### 🖼 **Video Inference and Object Detection**
- [x] Load the fine-tuned YOLOv8 model

### 🚗 **Integrate Deep SORT for Object Tracking**

- [x] Draw bounding boxes and track IDs on the video frames using OpenCV.

### 🔄 **Real-Time Video Processing (Optional)**

### 🎥 **Saving Output Video (Optional)**

### ⚙️ **Additional Enhancements**
- [x] Tune **Deep SORT parameters** (e.g., `max_age`, `min_confidence`) for improved tracking performance.
- [ ] Implement **NMS (Non-Maximum Suppression)** to handle overlapping bounding boxes if necessary.
- [x] Add vehicle **speed estimation** or **counting** if required for your project.

---

In [ ]:
!pip install ultralytics
!pip install deep-sort-realtime

In [ ]:
from ultralytics import YOLO
import cv2
from deep_sort_realtime.deepsort_tracker import DeepSort

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
fine_tuned_model = "/content/drive/MyDrive/Deep_Learning/Vehicle_Tracking/saved_model.pt"
model = YOLO(fine_tuned_model)

In [ ]:
!unzip "/content/drive/MyDrive/Deep_Learning/Vehicle_Tracking/CarsTrucksDetection.v6i.yolov8.zip"

In [ ]:
model.train(
    data="/content/data.yaml",
    epochs=50,
    imgsz=640,
    cos_lr=True,
    batch=64,
    device=0
)

```bash
50 epochs completed in 0.312 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.101 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.08it/s]
                   all        100        294      0.798      0.788      0.837      0.533
                   Car         43        142      0.758       0.81      0.825      0.499
                 Plate         37         48      0.747      0.708      0.772      0.438
                 Truck         74        104      0.889      0.846      0.913      0.662
Speed: 0.2ms preprocess, 2.4ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/train
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d5dec0e6680>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
```

In [ ]:
model.val()

```bash

Ultralytics YOLOv8.2.101 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /content/valid/labels.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.43s/it]
                   all        100        294      0.796      0.789      0.836      0.532
                   Car         43        142      0.755       0.81      0.825      0.495
                 Plate         37         48      0.743      0.708      0.771      0.439
                 Truck         74        104      0.889      0.848      0.913      0.663
Speed: 0.3ms preprocess, 4.9ms inference, 0.0ms loss, 3.8ms postprocess per image
Results saved to runs/detect/train2
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d5df2578910>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
```

In [ ]:
def network_inference(input_path: str = "inference_video.mp4",
                      output_path: str = "output_video5.mp4"):
    tracker = DeepSort(max_age=30, n_init=3)

    cap = cv2.VideoCapture(input_path)

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    model_input_size = (640, 640)

    while True:
        isTrue, frame = cap.read()
        if not isTrue:
            break

        resized_frame = cv2.resize(frame, model_input_size, interpolation=cv2.INTER_LINEAR)

        results = model(resized_frame)

        boxes = results[0].boxes.xyxy
        class_ids = results[0].boxes.cls
        confidences = results[0].boxes.conf

        detections = []
        car_count = 0
        truck_count = 0

        for i, box in enumerate(boxes):
            confidence = confidences[i]
            class_id = int(class_ids[i])

            if confidence < 0.5:
                continue

            if class_id == 0:
                car_count += 1
            elif class_id == 2:
                truck_count += 1

            x1, y1, x2, y2 = map(int, box)
            bbox = [x1, y1, x2 - x1, y2 - y1]
            detections.append((bbox, confidence, class_id))

        tracks = tracker.update_tracks(detections, frame=resized_frame)

        for track in tracks:
            if not track.is_confirmed():
                continue

            track_id = track.track_id
            ltrb = track.to_ltwh()
            x1, y1, w, h = map(int, ltrb)

            class_id = track.det_class
            color = (0, 255, 0) if class_id == 0 else (255, 0, 0)

            cv2.rectangle(resized_frame, (x1, y1), (x1 + w, y1 + h), color, 2)

            cv2.putText(resized_frame, f"ID {track_id} | {confidence:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

        text = f"Cars: {car_count} | Trucks: {truck_count}"
        cv2.putText(resized_frame, text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        output_frame = cv2.resize(resized_frame, (frame_width, frame_height), interpolation=cv2.INTER_LINEAR)

        out.write(output_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    print(f"Video saved to {output_path}")


In [ ]:
network_inference()